In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
from scipy.stats import pearsonr

df = pd.read_csv('D:/DSL/EDA/Data/고등학교_데이터셋.csv', encoding = 'cp949')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 46 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   학교_식별자          2349 non-null   object 
 1   소재지지번주소         2349 non-null   object 
 2   개방ID            2349 non-null   int64  
 3   위도              2349 non-null   float64
 4   경도              2349 non-null   float64
 5   시군구별            2331 non-null   object 
 6   2009            2349 non-null   int64  
 7   2010            2349 non-null   int64  
 8   2011            2349 non-null   int64  
 9   2012            2349 non-null   int64  
 10  2013            2349 non-null   int64  
 11  2014            2349 non-null   int64  
 12  2015            2349 non-null   int64  
 13  2016            2349 non-null   int64  
 14  2017            2349 non-null   int64  
 15  2018            2349 non-null   int64  
 16  2019            2349 non-null   int64  
 17  2020            2349 non-null   i

PCC가 0.8 이상인 행들만 걸러내기

In [2]:
train_X = np.array(df.iloc[:, 21:30])
train_y = np.array(df.iloc[:, 12:21])

def rowwise_correlation(A, B):
    """
    각 행별로 Pearson 상관 계수를 계산하여 반환하는 함수
    :param A: (n, m) 크기의 numpy 배열
    :param B: (n, m) 크기의 numpy 배열
    :return: (n,) 크기의 numpy 배열 (각 행의 Pearson 상관 계수)
    """
    return np.array([pearsonr(A[i], B[i])[0] for i in range(A.shape[0])])

corr = rowwise_correlation(train_X, train_y)
threshold = 0.8

# 원래 df 데이터에 대한 corr >= 0.8인 인덱스들
indices = np.where(corr >= threshold)[0]

# corr >= 0.8인 데이터들에 대해 새로운 데이터프레임인 df_new를 생성
df_new = df.iloc[indices]

train_X = df_new.iloc[:, 21:30]
train_y = df_new.iloc[:, 12:21]
test_X = df_new.iloc[:, 30:]

C:\Users\Monet\AppData\Local\Temp\ipykernel_8132\835209311.py:11: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return np.array([pearsonr(A[i], B[i])[0] for i in range(A.shape[0])])


뽑아낸 train_X와 train_y를 바탕으로 모델 학습시키기

In [3]:
train_X = np.array(train_X.fillna(0))
train_y = np.array(train_y.fillna(0))
test_X = np.array(test_X.fillna(0))

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression

class ZeroIgnoringLinearRegression:
    def __init__(self):
        """0을 무시하는 선형 회귀 모델"""
        self.model = LinearRegression()

    def fit(self, X, y):
        """
        0이 포함된 데이터를 제거하고 선형 회귀 모델 학습
        :param X: (n, m) 크기의 입력 특성 배열 (numpy 배열)
        :param y: (n,) 크기의 타겟 배열 (numpy 배열)
        """
        # X와 y에서 0이 포함된 행을 제거
        mask = (X != 0).all(axis=1) & (y != 0)
        X_filtered = X[mask]
        y_filtered = y[mask]

        # 선형 회귀 모델 학습
        self.model.fit(X_filtered, y_filtered)

    def predict(self, X):
        """
        주어진 데이터를 기반으로 예측 수행
        :param X: (n, m) 크기의 입력 특성 배열
        :return: (n,) 크기의 예측값 배열
        """
        return self.model.predict(X)

    def score(self, X, y):
        """
        모델의 R^2 점수 계산 (평가할 때는 0 포함)
        :param X: (n, m) 크기의 입력 특성 배열
        :param y: (n,) 크기의 타겟 배열
        :return: R^2 점수
        """
        return self.model.score(X, y)



In [ ]:
#train_X, train_y = np.array(train_X), np.array(train_y)

pred = []


for x, y, z in zip(train_X, train_y, test_X):
    x,y = x.reshape(-1,1), y.reshape(-1,1)
    tmp = LinearRegression()
    tmp.fit(x,y)
    
    pred.append(tmp.predict(z.reshape(-1,1)).T)


data = np.concat(pred, axis = 0)

print(data.shape)

IndexError: boolean index did not match indexed array along axis 1; size of axis is 1 but size of corresponding boolean axis is 9

위에서 구한 데이터를 새로운 pandas dataframe으로 바꿔주기

In [164]:
new_cols = [f'{year}년_예측_학생수' for year in range(2024, 2040)]
tmp_df = pd.DataFrame(data, columns = new_cols)

tmp_df.index = df_new.index
df_final = pd.concat([df_new, tmp_df], axis=1)

,2024년_예측_학생수,2025년_예측_학생수,2026년_예측_학생수,2027년_예측_학생수,2028년_예측_학생수,2029년_예측_학생수,2030년_예측_학생수,2031년_예측_학생수,2032년_예측_학생수,2033년_예측_학생수,2034년_예측_학생수,2035년_예측_학생수,2036년_예측_학생수,2037년_예측_학생수,2038년_예측_학생수,2039년_예측_학생수
0,4.626093,4.243345,3.823255,3.879267,4.411380,4.019297,3.505854,2.562987,2.049545,1.293384,0.546558,0.406528,0.602570,0.098463,-0.564345,-1.329841
1,785.061517,811.077220,832.377410,847.890378,861.179553,840.093705,824.741493,799.020509,786.615493,759.152984,723.465120,694.636184,662.886845,633.468470,602.924802,580.874417
2,922.834328,911.995953,893.405628,910.488630,1018.728822,1096.032992,1151.086194,1140.822037,1111.752224,1021.169251,956.354335,910.919293,889.673208,834.117565,791.912503,796.075587
3,485.556909,483.029935,463.408725,462.591174,484.888004,470.023451,450.625209,417.477255,389.383249,334.458724,259.318406,185.218607,124.273938,90.531402,75.889817,86.815263
6,52.607780,55.008243,56.219486,59.500854,61.725136,61.791204,59.985351,58.267588,55.955215,52.850029,47.894944,45.054028,43.666604,43.644582,42.609520,40.120966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2343,718.331463,681.891704,623.974207,589.103047,599.238609,571.607137,533.840102,500.537542,468.200272,405.818302,309.168478,248.958414,198.883911,162.564814,113.214280,71.827402
2344,1269.477632,1269.858547,1268.307678,1273.069117,1287.081352,1279.327008,1276.415728,1268.035596,1256.363268,1223.767817,1178.738208,1141.381317,1111.452272,1091.427020,1070.939228,1069.306735
2345,439.937562,426.453230,405.021179,392.117298,395.867907,385.643033,371.667550,359.344121,347.377893,324.293788,288.529053,266.248650,247.718857,234.279175,216.017281,200.702295
2347,880.013205,898.820171,912.390331,939.758336,974.548946,968.947840,862.071451,748.318908,649.320497,630.331381,599.138230,573.090810,555.149056,545.404042,535.659028,533.336618


In [162]:
df_final.to_csv('D:/DSL/EDA/Data/고등학교_학생수_예측.csv', index = False, sep = ',', encoding = 'utf-8-sig')